In [1]:
import pyspark
import os
#os.environ['PYSPARK_PYTHON'] = r'C:\Users\abhi3\Documents\work\ds-algo\pyspark_practice\spark-3.5.3-bin-hadoop3\spark-3.5.3-bin-hadoop3\python'
os.environ["SPARK_HOME"]=r"C:\Users\abhi3\Documents\work\ds-algo\pyspark_practice\spark-3.5.3-bin-hadoop3\spark-3.5.3-bin-hadoop3"
#os.environ["PYTHONPATH"]=r'$(ZIPS=("$SPARK_HOME"/python/lib/.zip); IFS=:; echo "${ZIPS[]}"):$PYTHONPATH'
os.environ["JAVA_HOME"]=r'C:\Program Files\Java\jre1.8.0_431'
os.environ["HADOOP_HOME"]=r"C:\Users\abhi3\Documents\work\ds-algo\pyspark_practice\spark-3.5.3-bin-hadoop3\spark-3.5.3-bin-hadoop3\hadoop"
os.environ["PYSPARK_DRIVER_PYTHON"]="jupyter"
postgresjdbc_path=r"C:\Users\abhi3\Documents\work\ds-algo\sql_practice\postgresql-42.7.2.jar"

In [2]:
from pyspark.sql import SparkSession
from datetime import datetime, date
#import pandas as pd
from pyspark.sql import Row
spark = (SparkSession
         .builder
         .appName("MyApp")
         #.config("spark.jars", f"{postgresjdbc_path}")
         .config("spark.driver.extraClassPath", postgresjdbc_path)
         .getOrCreate())

In [28]:
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [4]:
def publicis_test(text_path,columns):
    file_read=spark.read.text(text_path)
    index_rdd=file_read.rdd.map(lambda x: x.value.split()) #to split the elements
    index_rdd=index_rdd.filter(lambda x:x) #to remove the null
    index_rdd=index_rdd.toDF(columns)
    return index_rdd
text_path=r"C:\Users\abhi3\Documents\work\ds-algo\interview_test\publicis_sapient.txt"
df_rdd=publicis_test(text_path,["date","time","category"])
df_rdd.collect()

[Row(date='2023-01-01', time='1:05:00', category='u1'),
 Row(date='2023-01-01', time='1:25:00', category='u1'),
 Row(date='2023-01-02', time='2:05:00', category='u1'),
 Row(date='2023-01-02', time='2:35:00', category='u1'),
 Row(date='2023-01-02', time='3:05:00', category='u1'),
 Row(date='2023-01-12', time='1:05:00', category='u2'),
 Row(date='2023-01-12', time='1:05:00', category='u2'),
 Row(date='2023-01-12', time='1:05:00', category='u2'),
 Row(date='2023-01-01', time='1:05:00', category='u1'),
 Row(date='2023-01-12', time='1:35:00', category='u2')]

In [75]:
df_rdd.filter(df_rdd["category"]=="u1").collect()

[Row(date='2023-01-01', time='1:05:00', category='u1'),
 Row(date='2023-01-01', time='1:25:00', category='u1'),
 Row(date='2023-01-02', time='2:05:00', category='u1'),
 Row(date='2023-01-02', time='2:35:00', category='u1'),
 Row(date='2023-01-02', time='3:05:00', category='u1'),
 Row(date='2023-01-01', time='1:05:00', category='u1')]

In [79]:
df_rdd.filter(df_rdd["time"]>"2:00:00").collect()

[Row(date='2023-01-02', time='2:05:00', category='u1'),
 Row(date='2023-01-02', time='2:35:00', category='u1'),
 Row(date='2023-01-02', time='3:05:00', category='u1')]

In [7]:
from pyspark.sql.functions import to_timestamp, col,concat_ws
#df_rdd_1=df_rdd.select(concat(col["date"]," ",col["time"]).alias("date_time"))#withColumn("date_time",concatenate(col["date"]," ",col["time"]))#to_timestamp("time","hh:mm:ss"))
df_rdd_1=df_rdd.select(concat_ws(" ",df_rdd.date,df_rdd.time).alias("date_time"))#withColumn("date_time",concatenate(col["date"]," ",col["time"]))#to_timestamp("time","hh:mm:ss"))

df_rdd_1.show()

+------------------+
|         date_time|
+------------------+
|2023-01-01 1:05:00|
|2023-01-01 1:25:00|
|2023-01-02 2:05:00|
|2023-01-02 2:35:00|
|2023-01-02 3:05:00|
|2023-01-12 1:05:00|
|2023-01-12 1:05:00|
|2023-01-12 1:05:00|
|2023-01-01 1:05:00|
|2023-01-12 1:35:00|
+------------------+



In [ ]:
df_rdd_1=df_rdd.withColumns({"date_time":concat_ws(" ",df_rdd.date,df_rdd.time)})
df_rdd_1.collect()

[Row(date='2023-01-01', time='1:05:00', category='u1', date_time='2023-01-01 1:05:00'),
 Row(date='2023-01-01', time='1:25:00', category='u1', date_time='2023-01-01 1:25:00'),
 Row(date='2023-01-02', time='2:05:00', category='u1', date_time='2023-01-02 2:05:00'),
 Row(date='2023-01-02', time='2:35:00', category='u1', date_time='2023-01-02 2:35:00'),
 Row(date='2023-01-02', time='3:05:00', category='u1', date_time='2023-01-02 3:05:00'),
 Row(date='2023-01-12', time='1:05:00', category='u2', date_time='2023-01-12 1:05:00'),
 Row(date='2023-01-12', time='1:05:00', category='u2', date_time='2023-01-12 1:05:00'),
 Row(date='2023-01-12', time='1:05:00', category='u2', date_time='2023-01-12 1:05:00'),
 Row(date='2023-01-01', time='1:05:00', category='u1', date_time='2023-01-01 1:05:00'),
 Row(date='2023-01-12', time='1:35:00', category='u2', date_time='2023-01-12 1:35:00')]

In [3]:
#df_rdd_1.withColumns({"date_time_1":to_timestamp(col("date_time"),"yyyy-MM-dd HH:mm:ss")}).collect()

In [6]:
text_path=r"C:\Users\abhi3\Documents\work\ds-algo\pyspark_practice\test.txt"
df=publicis_test(text_path,['product_id', 'new_price', 'change_date'])
df.show()

+----------+---------+-----------+
|product_id|new_price|change_date|
+----------+---------+-----------+
|product_id|new_price|change_date|
|         1|       20| 2019-08-14|
|         2|       50| 2019-08-14|
|         1|       30| 2019-08-15|
|         1|       35| 2019-08-16|
|         2|       65| 2019-08-17|
|         3|       20| 2019-08-18|
+----------+---------+-----------+



In [5]:
text_path=r"C:\Users\abhi3\Documents\work\ds-algo\pyspark_practice\test.txt"
file_read=spark.read.text(text_path)
file_read

DataFrame[value: string]

In [6]:
file_read.collect()

[Row(value=' product_id  new_price  change_date '),
 Row(value=' 1           20         2019-08-14  '),
 Row(value=' 2           50         2019-08-14  '),
 Row(value=' 1           30         2019-08-15  '),
 Row(value=' 1           35         2019-08-16  '),
 Row(value=' 2           65         2019-08-17  '),
 Row(value=' 3           20         2019-08-18  ')]

In [9]:
def func_split(x):
    #print(x)
    return x.value.split()

index_rdd=file_read.rdd.map(lambda x: x.value.split())#.zipWithIndex()
(index_rdd.collect())
#.toDF(["date","date_time","category"]).collect()

[['product_id', 'new_price', 'change_date'],
 ['1', '20', '2019-08-14'],
 ['2', '50', '2019-08-14'],
 ['1', '30', '2019-08-15'],
 ['1', '35', '2019-08-16'],
 ['2', '65', '2019-08-17'],
 ['3', '20', '2019-08-18']]

In [8]:
index_rdd_1=index_rdd.filter(lambda x: x)
index_rdd_1.collect()

[['product_id', 'new_price', 'change_date'],
 ['1', '20', '2019-08-14'],
 ['2', '50', '2019-08-14'],
 ['1', '30', '2019-08-15'],
 ['1', '35', '2019-08-16'],
 ['2', '65', '2019-08-17'],
 ['3', '20', '2019-08-18']]

In [11]:
column_name=index_rdd_1.first()
column_name


['product_id', 'new_price', 'change_date']

In [15]:
index_rdd_1.filter(lambda x:x!=column_name)
.collect()

[['1', '20', '2019-08-14'],
 ['2', '50', '2019-08-14'],
 ['1', '30', '2019-08-15'],
 ['1', '35', '2019-08-16'],
 ['2', '65', '2019-08-17'],
 ['3', '20', '2019-08-18']]

In [17]:
#.zipWithIndex()
index_rdd_2=index_rdd_1.filter(lambda x:x!=column_name).toDF(column_name)
index_rdd_2.collect()

[Row(product_id='1', new_price='20', change_date='2019-08-14'),
 Row(product_id='2', new_price='50', change_date='2019-08-14'),
 Row(product_id='1', new_price='30', change_date='2019-08-15'),
 Row(product_id='1', new_price='35', change_date='2019-08-16'),
 Row(product_id='2', new_price='65', change_date='2019-08-17'),
 Row(product_id='3', new_price='20', change_date='2019-08-18')]

In [18]:
index_rdd_2.show()

+----------+---------+-----------+
|product_id|new_price|change_date|
+----------+---------+-----------+
|         1|       20| 2019-08-14|
|         2|       50| 2019-08-14|
|         1|       30| 2019-08-15|
|         1|       35| 2019-08-16|
|         2|       65| 2019-08-17|
|         3|       20| 2019-08-18|
+----------+---------+-----------+

